In [8]:
#Some work into the actual gamestates of the problems
#looked at stockfish position ratings of game tree to try to predict problem difficulty

import sys
import re
import pandas as pd
import os
import sys
import chess
import chess.engine
import pystockfish as ps
from multiprocessing import Pool
import resource
import matplotlib
#%pylab inline
soft, hard = resource.getrlimit(resource.RLIMIT_NOFILE)
resource.setrlimit(resource.RLIMIT_NOFILE, (hard, hard))

idsToSearchFor = [39943, 36197, 43124, 30514, 39335, 35847, 38567, 35865,40486]

In [9]:
import numpy as np

def gini(array):
    print(array)
    """Calculate the Gini coefficient of a numpy array."""
    # based on bottom eq:
    # http://www.statsdirect.com/help/generatedimages/equations/equation154.svg
    # from:
    # http://www.statsdirect.com/help/default.htm#nonparametric_methods/gini.htm
    # All values are treated equally, arrays must be 1d:
    array = array.flatten()
    array = [float(v) for v in array]
    if np.amin(array) < 0:
        # Values cannot be negative:
        array -= np.amin(array)
    # Values cannot be 0:
    print(array)
    
    array += 0.0000001
    # Values must be sorted:
    array = np.sort(array)
    # Index per array element:
    index = np.arange(1,array.shape[0]+1)
    # Number of array elements:
    n = array.shape[0]
    # Gini coefficient:
    return ((np.sum((2 * index - n - 1) * array)) / (n * np.sum(array)))

In [10]:
path = '''/w/225/1/chess/tactics/'''
os.chdir(path)
filename1 = '''tactics_problem.csv'''
filename2= '''glicko_user_tactics_problem.csv_00'''

In [11]:
engine_location = "/h/224/subramanian/stockfish-10-linux/Linux/stockfish-10-linux/Linux/stockfish_10_x64"


class Node():
    def __init__(self, FEN, solution = False, depths = [0, 1, 2, 5, 10]):
        #FEN is  a string with the FEN of the position
        #Solution is either False of a list of moves in SAN format.
        #Depths is a list of depths to check for each position
        self.board = chess.Board(fen=FEN)
        self.moves = self.board.legal_moves
        self.depths = depths
        self.solution = solution
        self.has_solution = bool(solution)
        self.move_score_dict = {}
        self.to_move = self.board.turn
        
        for d in depths:
            self.move_score_dict[d] = {}
    
    def get_scores(self):
        if self.to_move:
            multiplier = -1
        else:
            multiplier = 1
        #engines = {}
        
        
        #print("solution", self.solution)
        self.score_dict = {}
        self.solution_dict = {}
        #pool = Pool(processes=5)
        #output = pool.map(self.helper_function,self.depths)
        #print(output)
        #sys.exit()
        #move_dict = {}
        #print(self.solution)
        #print(self.board.fen())
        #sys.exit()
        self.engine = chess.engine.SimpleEngine.popen_uci(engine_location)
        engine = self.engine
        for d in self.depths:
            self.score_dict[d] = []
            #print(self.board)
            self.board.push(self.solution)
            score = str(engine.analyse(self.board, chess.engine.Limit(depth=d))["score"])
            self.board.pop()
            if score[0] != "#":
                mate = False
                if int(score) >= 3000:
                    score = 2999
                elif int(score) <= -3000:
                    score = -2999
                else:
                    score = int(score)
            elif int(score[1:]) > 0:
                mate = True
                score = 3000
            else:
                mate = True
                score = -3000
            self.solution_dict[d] = score * multiplier
                
            #self.board.pop()
            for move in self.moves:
                if move != self.solution:
                    self.board.push(move)
                    info = engine.analyse(self.board, chess.engine.Limit(depth=d))
                    score = str(info["score"])
                    if score[0] != "#":
                        mate = False
                        if int(score) >= 3000:
                            score = 2999
                        elif int(score) < -2999:
                            score = -2999
                        else:
                            score = int(score)
                    elif int(score[1:]) > 0:
                        mate = True
                        score = 3000
                    else:
                        mate = True
                        score = -3000
                    self.score_dict[d].append(score * multiplier)
                    #print("move", move, "Score:", info["score"], "depth", d, "turn", self.board.turn)
                    self.board.pop()
                    #print(move, score, d)
                    #if str(move) in move_dict:
                    #    move_dict[str(move)].append(score * multiplier)
                    #else:
                    #    move_dict[str(move)] = [score * multiplier]
        #for d in self.depths:
        #    print(d)
        #    print(score_dict[d])
        #print(solution_dict)
        #print(score_dict)
        
        #print([(d, gini(np.array(score_dict[d]))) for d in score_dict])
        towrite2 = str(self.to_move) + "," + str(self.solution_dict[1])+ "," + str(self.solution_dict[2])+ "," + str(self.solution_dict[5])+ "," + str(self.solution_dict[10]) +"," + str(max(self.score_dict[1]))+"," + str(max(self.score_dict[2]))+"," + str(max(self.score_dict[5]))+"," + str(max(self.score_dict[10])) + "\n"#"," + str(gini(np.array(score_dict[1]))) + "," + str(gini(np.array(score_dict[2]))) + "," + str(gini(np.array(score_dict[5]))) + "," + str(gini(np.array(score_dict[10]))) + "\n"
        return towrite2
        # Score: #1
        #for d in self.depths
        
        #    engine = ps.Engine(depth=d, rand=False)
            #for move in self.moves:'''
                
        ###################################################################################################################################
        
    
    def get_all_depth_1(self):
        rows = []
        if self.to_move:
            multiplier = -1
        else:
            multiplier = 1
        #engines = {}
        
        
        #print("solution", self.solution)
        self.score_dict = {}
        self.solution_dict = {}
        #pool = Pool(processes=5)
        #output = pool.map(self.helper_function,self.depths)
        #print(output)
        #sys.exit()
        #move_dict = {}
        #print(self.solution)
        #print(self.board.fen())
        #sys.exit()
        self.engine = chess.engine.SimpleEngine.popen_uci(engine_location)
        engine = self.engine
        for d in self.depths:
            self.score_dict[d] = []
            #print(self.board)
            self.board.push(self.solution)
            score = str(engine.analyse(self.board, chess.engine.Limit(depth=d))["score"])
            self.board.pop()
            if score[0] != "#":
                mate = False
                if int(score) >= 3000:
                    score = 2999
                elif int(score) <= -3000:
                    score = -2999
                else:
                    score = int(score)
            elif int(score[1:]) > 0:
                mate = True
                score = 3000
            else:
                mate = True
                score = -3000
            self.solution_dict[d] = score * multiplier
            
            row = str(self.to_move) + "," + str(self.solution) + "," + str(d) + "," + str(score * multiplier) +","+ str(score)+",1\n"
            rows.append(row)
                
            #self.board.pop()
            for move in self.moves:
                if move != self.solution:
                    self.board.push(move)
                    info = engine.analyse(self.board, chess.engine.Limit(depth=d))
                    score = str(info["score"])
                    if score[0] != "#":
                        mate = False
                        if int(score) >= 3000:
                            score = 2999
                        elif int(score) < -2999:
                            score = -2999
                        else:
                            score = int(score)
                    elif int(score[1:]) > 0:
                        mate = True
                        score = 3000
                    else:
                        mate = True
                        score = -3000
                        
                    #isWhite,move,depth,score,isSolution\
                    
                    row = str(self.to_move) + "," + str(move) + "," + str(d) + "," + str(score * multiplier) +","+ str(score)+",0\n"
                    rows.append(row)
                    self.score_dict[d].append(score * multiplier)
                    #print("move", move, "Score:", info["score"], "depth", d, "turn",  self.board.turn)
                    self.board.pop()
                    #print(move, score, d)
                    #if str(move) in move_dict:
                    #    move_dict[str(move)].append(score * multiplier)
                    #else:
                    #    move_dict[str(move)] = [score * multiplier]
        #for d in self.depths:
        #    print(d)
        #    print(score_dict[d])
        #print(solution_dict)
        #print(score_dict)
        
        #print([(d, gini(np.array(score_dict[d]))) for d in score_dict])
        #towrite2 = str(self.to_move) + "," + str(self.solution_dict[1])+ "," + str(self.solution_dict[2])+ "," + str(self.solution_dict[5])+ "," + str(self.solution_dict[10]) +"," + str(max(self.score_dict[1]))+"," + str(max(self.score_dict[2]))+"," + str(max(self.score_dict[5]))+"," + str(max(self.score_dict[10])) + "\n"#"," + str(gini(np.array(score_dict[1]))) + "," + str(gini(np.array(score_dict[2]))) + "," + str(gini(np.array(score_dict[5]))) + "," + str(gini(np.array(score_dict[10]))) + "\n"
        return rows
        # Score: #1
        #for d in self.depths
        
        #    engine = ps.Engine(depth=d, rand=False)
            #for move in self.moves:'''
                
                
    
    
       
       

In [ ]:
pattern = r'''FEN "[^"]*'''
pattern_solution = r'''FULL "[^"]*'''

f3 = "positional_data_no_depth_5.csv"
f4 = "positional_data_just_depth_5.csv"
#board = chess.Board("r3k2r/ppp1b2p/2n1pNp1/1Q1q4/2pp4/8/PPP2PPP/R1B1R1K1 b kq - 1 1")

#print(board)
#sys.exit()

positional_csv = open(f3, "w")

#toWrite = "tactics_problem_id,move_count,solution,piece_to_move,forcing_move,num_legal_moves,isWhite,move,depth,score,score_no_multiplier,isSolution\n"#scoreDepth1,scoreDepth2,scoreDepth5,scoreDepth10,bestAlt1,bestAlt2,bestAlt5,bestAlt10\n"#,gini1,gini2,gini5,gini10\n"

toWrite = "tactics_problem_id,move_count,solution,piece_to_move,forcing_move,num_legal_moves\n"#,isWhite,move,depth,score,score_no_multiplier,isSolution\n"#scoreDepth1,scoreDepth2,scoreDepth5,scoreDepth10,bestAlt1,bestAlt2,bestAlt5,bestAlt10\n"#,gini1,gini2,gini5,gini10\n"

positional_csv.write(toWrite)
positional_csv.close()
toWrite2 = "tactics_problem_id,isWhite,move,depth,score,score_no_multiplier,isSolution\n"

positional_csv_2 = open(f4, "w")


positional_csv_2.write(toWrite2)
positional_csv_2.close()

toWrite = ""
toWrite2 = ""

i = 0
errors = []
#attempts = []
#positional_csv = open(f3, "a")
j = 0
for chunk in pd.read_csv(filename1, chunksize=500): 
    #toWrite = ""
    #if i > 10030:
    #    sys.exit()
    

    
    for index, row in chunk.iterrows():
        #if i > 510:
        #    sys.exit()
        #print(index, row)
        #sys.exit()
        attempt_count = int(row["attempt_count"])
        #attempts.append(attempt_count)
        if attempt_count < 10000:# or attempt_count < 63984: #ranked 1000-2000
            continue
        #print(i)
        
        if i % 100 == 0:
            print(i)
        i = i + 1
        
        if i < 11000:
            continue
        
        
        
        
        
        problem_id = row["tactics_problem_id"]
        pgn = row["pgn"]
        #print(pgn)
        
        #Get FEN and board
        FEN = re.findall(pattern, pgn)[0][5:]
        #print(FEN)
        #print(FEN)
        if len(FEN.split()) != 6:
            #print(FEN, len(FEN.split()))
            FEN = FEN + " 1 1"
        
        try:
            board = chess.Board(fen=FEN)
        except:
            errors.append([problem_id, FEN])
            #print(errors[-1], 0)
            continue
                        
            
            #print(pgn)
            #sys.exit()
            #board = chess.Board(fen=FEN)
        #print(board)
        
        #get move count
        move_count = row["move_count"]
        #print(move_count)

        #get solution
        #p = False
        try:
            solution = re.findall(pattern_solution, pgn)[0][6:]
            solution = solution.split(" ")
        except:
            solution = pgn.split("]")[-1]
            #print(solution)
            if solution[-1] == "*":
                solution = solution[4:-2]
            else:
                solution = solution[4:-4]
            #print(solution)
            if "(" in solution: #there's literally one instance of this. 
                solution = re.sub('''\([^)]*\)''', "", solution)
                #print(solution)
            solution = re.split('\.| ',solution)
            #p = True
            #print(solution)
            #sys.exit()
            
            #errors.append([problem_id, FEN])
            #print(pgn, 1)
            #continue
        #print(solution)
        
        solution2 = []
        for move in solution:
            if len(move)>1 and move[0].isalpha():
                solution2.append(move)
        solution, solution2 = solution2, None
        #for move in solution:
        board.push_san(solution[0])
        #    break
            #print("\n", board)
        solution = solution[1:]
        #if p:
        #    print(solution)
        #    print("\n")
        #print(" ".join(solution))
        
        try:
            next_move = board.parse_san(solution[0])
        except:
            print(pgn)
            print(FEN)
            print(solution)
            print("error with above\n\n")
            continue
        #print(solution, next_move, board.turn)
        #print(next_move)
        #print(next_move.from_square)
        #print(next_move.to_square)
        
        ###
        #if move_count < 6:
        #    continue
        #print(board.fen(), solution)
        #try:
        
        
        
        board2 = Node(board.fen(), solution = next_move, depths = [1, 2, 5, 10])
        
        
        try:
            list_to_write = board2.get_all_depth_1()
        except:
            continue
            print("error for pgn", pgn)
        
        
        #except:
        #    print(pgn)
        #    continue
        #sys.exit()
        
        ###
        
        #Get piece to move
        piece_to_move = board.piece_at(next_move.from_square)
        #sys.exit()
        
        #print(board.fen())
        #get if forcing move
        forcing_move = False
        if "+" in solution[0] or "#" in solution[0] or "x" in solution[0]:
            forcing_move = True
        #print(forcing_move)
        
        #Get possible moves, num possible moves
        possible_moves = board.legal_moves
        num_legal_moves = possible_moves.count()
        #print(num_legal_moves)
        #for m in possible_moves:
        #    print(m)
        
        for row in list_to_write:
            toWrite2 = toWrite2 +  str(problem_id) + "," + row
            
        toWrite = toWrite + str(problem_id) + "," + str(move_count) + "," + " ".join(solution) + "," + str(piece_to_move) + "," + str(int(forcing_move)) + "," + str(num_legal_moves) + "\n"# + row
        #print(toWrite)
        #sys.exit()
    
    #print(toWrite)
    positional_csv = open(f3, "a")

    positional_csv.write(toWrite)
    toWrite = ""
    positional_csv.close()
    
    positional_csv2 = open(f4, "a")

    positional_csv2.write(toWrite2)
    toWrite2 = ""
    positional_csv2.close()

    print("written")
positional_csv.close()
    #print("chunk done")
print("written")
        


0
100
200
300
written
400
500
600
700
written
800
900
1000
1100
written
1200
1300
1400
1500
written
1600
1700
1800
1900
written
2000
2100
2200
2300
written
2400
2500
2600
written
2700
2800
2900
3000
written
3100
3200
3300
3400
written
3500
3600
3700
3800
written
3900
4000
4100
4200
written
4300
4400
4500
4600
written
4700
4800
4900
5000
written
5100
5200
5300
5400
written
5500
5600
5700
written
5800
5900
6000
6100
written
6200
6300
6400
6500
written
6600
6700
6800
written
6900
7000
7100
written
7200
7300
7400
written
7500
7600
7700
written
7800
7900
8000
written
8100
8200
8300
written
8400
8500
8600
written
8700
8800
8900
written
9000
9100
9200
written
9300
9400
9500
written
9600
9700
9800
written
9900
10000
10100
written
10200
10300
10400
written
10500
10600
10700
written
10800
10900
11000
written
11100
11200
11300
written
11400
11500
11600
written
11700
11800
11900
written
12000
12100
12200
written
12300
12400
12500
written
12600
12700
12800
written
12900
13000
written
13100
13200
13

In [186]:
attempts.sort(reverse = True)
attempts[1000:2000]

[64826,
 64824,
 64823,
 64823,
 64821,
 64820,
 64819,
 64819,
 64818,
 64818,
 64815,
 64814,
 64814,
 64813,
 64811,
 64810,
 64810,
 64810,
 64809,
 64809,
 64808,
 64808,
 64804,
 64803,
 64803,
 64802,
 64802,
 64802,
 64801,
 64800,
 64800,
 64800,
 64800,
 64799,
 64796,
 64795,
 64795,
 64794,
 64793,
 64792,
 64792,
 64792,
 64790,
 64790,
 64787,
 64786,
 64785,
 64783,
 64780,
 64778,
 64777,
 64776,
 64775,
 64774,
 64772,
 64769,
 64768,
 64767,
 64764,
 64763,
 64760,
 64760,
 64760,
 64757,
 64757,
 64756,
 64754,
 64754,
 64753,
 64752,
 64751,
 64751,
 64748,
 64748,
 64748,
 64748,
 64746,
 64746,
 64745,
 64745,
 64745,
 64744,
 64744,
 64744,
 64744,
 64743,
 64741,
 64739,
 64738,
 64737,
 64736,
 64733,
 64733,
 64732,
 64730,
 64730,
 64729,
 64729,
 64729,
 64728,
 64728,
 64727,
 64726,
 64726,
 64726,
 64725,
 64725,
 64725,
 64724,
 64724,
 64724,
 64722,
 64722,
 64719,
 64718,
 64718,
 64716,
 64716,
 64714,
 64712,
 64712,
 64711,
 64708,
 64707,
 64707,


In [38]:
engine.quit()
engine = chess.engine.SimpleEngine.popen_uci(engine_location)
pattern = r'''FEN "[^"]*'''
pattern_solution = r'''FULL "[^"]*'''

In [13]:
largest = [0]
smallest = [10000]
for chunk in pd.read_csv(filename1, chunksize=2000): 
    for index, row in chunk.iterrows():

        move_count = int(row["move_count"])

        
        if move_count != 3:
            continue
        attempt_count = int(row["attempt_count"])
        #attempts.append(attempt_count)
        if attempt_count < 10000:# or attempt_count < 63984: #ranked 1000-2000
            continue
        rating = int(row["rating"])
        if rating < max(smallest) and rating > 500:
            if len(smallest) == 4:
                smallest.remove(max(smallest))
            smallest.append(rating)
        if rating > min(largest):
            if len(largest) == 4:
                largest.remove(min(largest))
            largest.append(rating)
        
        #if i > 510:
        #    sys.exit()
        #print(index, row)
        #sys.exit()
        
        continue
        
    break

0 tactics_problem_id                                                  731
rating                                                             1569
rd                                                                44.96
attempt_count                                                     53336
average_seconds                                                      41
move_count                                                            3
tags                  Attacking f7/f2,Attacking Castled King,Back Ra...
pgn                   [Event "?"]\n[Site "?"]\n[Date "????.??.??"]\n...
Name: 0, dtype: object


SystemExit: 

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:3275: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [20]:
boards = []
ratings = []
solutions = []
move_counts = []
pgns = []
ids = []

for chunk in pd.read_csv(filename1, chunksize=2000): 
    for index, row in chunk.iterrows():
        rating = int(row["rating"])
        problem_id = int(row["tactics_problem_id"])
        if problem_id not in idsToSearchFor:
            continue
        else:
            print("found one!")
        #if rating not in largest and rating not in smallest:
        #    continue
        problem_id = row["tactics_problem_id"]
        pgn = row["pgn"]
        #print(pgn)
        
        #Get FEN and board
        FEN = re.findall(pattern, pgn)[0][5:]
        #print(FEN)
        #print(FEN)
        if len(FEN.split()) != 6:
            #print(FEN, len(FEN.split()))
            FEN = FEN + " 1 1"
        
        try:
            board = chess.Board(fen=FEN)
        except:
            errors.append([problem_id, FEN])
            #print(errors[-1], 0)
            continue
                        
            
            #print(pgn)
            #sys.exit()
            #board = chess.Board(fen=FEN)
        #print(board)
        
        #get move count
        move_count = row["move_count"]
        #print(move_count)

        #get solution
        #p = False
        try:
            solution = re.findall(pattern_solution, pgn)[0][6:]
            solution = solution.split(" ")
        except:
            solution = pgn.split("]")[-1]
            #print(solution)
            if solution[-1] == "*":
                solution = solution[4:-2]
            else:
                solution = solution[4:-4]
            #print(solution)
            if "(" in solution: #there's literally one instance of this. 
                solution = re.sub('''\([^)]*\)''', "", solution)
                #print(solution)
            solution = re.split('\.| ',solution)
            #p = True
            #print(solution)
            #sys.exit()
            
            #errors.append([problem_id, FEN])
            #print(pgn, 1)
            #continue
        #print(solution)
        
        solution2 = []
        for move in solution:
            if len(move)>1 and move[0].isalpha():
                solution2.append(move)
        solution, solution2 = solution2, None
        #for move in solution:
        board.push_san(solution[0])
        #    break
            #print("\n", board)
        solution = solution[1:]
        #if p:
        #    print(solution)
        #    print("\n")
        #print(" ".join(solution))
        
        try:
            next_move = board.parse_san(solution[0])
        except:
            print(pgn)
            print(FEN)
            print(solution)
            print("error with above\n\n")
            continue
        boards.append(board)
        ratings.append(rating)
        solutions.append(solution)
        move_counts.append(move_count)
        pgns.append(pgn)
        ids.append(problem_id)
    

found one!
found one!
found one!
found one!
found one!
found one!
found one!
found one!


In [15]:
print(largest, smallest)

[0] [10000]


In [17]:
z = 0

In [23]:
print(len(boards))
print(boards[z])

print(ratings[z])# = []
print()
print(solutions[z])# = []
print()
print(move_counts[z])# = []
print()
print(pgns[z])
print("")
print(boards)

8
. . k r . b . .
p p p q . p . p
. . n . b p . .
. . . . . . . .
. . . P . . . .
. . . B B . . .
P Q P . N P r P
R . . . . R K .
2556

['Kxg2', 'Bh3+', 'Kg1', 'Qd5', 'f3']

3

[Event "?"]\n[Site "Budapest"]\n[Date "1921.??.??"]\n[Round "?"]\n[White "Balla, Z."]\n[Black "Sterk, K."]\n[Result "1-0"]\n[DELTA "-5.40"]\n[FEN "2kr1br1/pppq1p1p/2n1bp2/8/3P4/3BB3/PQP1NPPP/R4RK1 b - -"]\n[FirstMove "13b"]\n[FULL "13. ... Rxg2+ 14. Kxg2 Bh3+ 15. Kg1 Qd5 16. f3"]\n[MOVES "3"]\n[PlyCount "6"]\n[SCORE "3.72"]\n[SetUp "1"]\n\n 13.     ...   Rxg2+\n\t\t\t({24:2.66} 13. ... Rxg2+ 14. Kxg2 Nb4 15. Be4 Re8 16. Kg1 Bg4 17. f3  $18)\n\t\t\t({24:0.09} 13. ... Nb4 14. Bxh7 Rh8 15. Be4 Bd6 16. Bf4 f5 17. Bf3  $10)\n\t\t\t({24:0.3} 13. ... Bd6 14. Ng3 f5 15. Rfb1 b6 16. Qb5 Ne7 17. Qxd7+  $10)\n\t\t\t({24:0.39} 13. ... Kb8 14. Nf4 Bf5 15. Bb5 Qd6 16. c3 Bh6 17. Nh5  $14)\n\t\t\t({24:0.53} 13. ... f5 14. a3 Kb8 15. Rab1 Na5 16. g3 b6 17. Nf4  $14)\n 14.    Kxg2\n\t\t\t({24:2.85} 14. Kxg2 Nb4 15. Be4 Re8 16. K

In [25]:
def get_top_moves(board, d = 0):
    scores = []
    moves = []
    for move in board.legal_moves:
        #print(move)
        board.push(move)
        score = str(engine.analyse(board, chess.engine.Limit(depth=d))["score"])
        #score1 = str(engine.analyse(boards[z], chess.engine.Limit(depth=1))["score"])

        board.pop()
        

        if score[1:].isnumeric() or score.isnumeric():
            pass
        elif score[1] == "+":
            score = 3000
        elif score[1] == "-":
            score = -3000

        moves.append(str(move))
        scores.append(int(score))


        
    scores, moves = (list(t) for t in zip(*sorted(zip(scores, moves))))
    scores.reverse()
    moves.reverse()
    print(scores, moves)
    '''
    try:
        scores = scores[:3]
        moves = moves[:3]
    except:
        pass'''
    
    
    return scores,moves





#print(moves)
#print(scores)

In [18]:
this_board = boards[z]
tree = []
layer_one_scores, layer_one_moves = get_top_moves(this_board)
layer_two_scores = []
layer_two_moves = []
layer_two_prev_moves = []
for move in layer_one_moves:
    print(move)
    print(this_board)
    this_board.push(move)
    s, m = get_top_moves(this_board)
    this_board.pop()
    layer_two_scores.append(s)
    layer_two_moves.append(m)
    layer_two_prev_moves.append([move])
    
layer_three_scores = []
layer_three_moves = []
layer_three_prev_moves = []
for i in range(len(layer_two_moves)):
    move = layer_two_moves[i]
    for m in layer_two_prev_moves[i]:
        this_board.push(m)
    this_board.push(move)
    s, m = get_top_moves(this_board)
    this_board.pop()
    for m in layer_two_prev_moves[i]:
        this_board.pop(m)
    layer_three_scores.append(s)
    layer_three_moves.append(m)
    prev = layer_two_prev_moves[i]
    prev = prev + [move]
    layer_three_prev_moves.append(prev)
    
layer_four_scores = []
layer_four_moves = []
layer_four_prev_moves = []
for i in range(len(layer_three_moves)):
    move = layer_three_moves[i]
    for m in layer_three_prev_moves[i]:
        this_board.push(m)
    this_board.push(move)
    s, m = get_top_moves(this_board)
    this_board.pop()
    for m in layer_three_prev_moves[i]:
        this_board.pop(m)
    layer_four_scores.append(s)
    layer_four_moves.append(m)
    prev = layer_three_prev_moves[i]
    prev = prev + [move]
    layer_four_prev_moves.append(prev)
    
    
    
layer_five_scores = []
layer_five_moves = []
layer_five_prev_moves = []
for i in range(len(layer_four_moves)):
    move = layer_four_moves[i]
    for m in layer_four_prev_moves[i]:
        this_board.push(m)
    this_board.push(move)
    s, m = get_top_moves(this_board)
    this_board.pop()
    for m in layer_four_prev_moves[i]:
        this_board.pop(m)
    layer_five_scores.append(s)
    layer_five_moves.append(m)
    prev = layer_four_prev_moves[i]
    prev = prev + [move]
    layer_five_prev_moves.append(prev)

d8d1
. . . r r . k .
p p p . . p p p
. . . . . . . .
. . . . . Q . .
. . . . n . . .
. B . . . . . R
P P P . . P P .
. . . . . K . .


AttributeError: 'str' object has no attribute 'from_square'

In [26]:
z = 0
this_board = boards[z]
layer_one_scores, layer_one_moves = get_top_moves(this_board, 1)
print(layer_one_moves, layer_one_scores)
print(ratings[z])
print(ids[z])
print(this_board.fen())
print(solutions[z])

print(this_board)

[594, -179] ['g1h1', 'g1g2']
['g1h1', 'g1g2'] [594, -179]
2556
30514
2kr1b2/pppq1p1p/2n1bp2/8/3P4/3BB3/PQP1NPrP/R4RK1 w - - 0 2
['Kxg2', 'Bh3+', 'Kg1', 'Qd5', 'f3']
. . k r . b . .
p p p q . p . p
. . n . b p . .
. . . . . . . .
. . . P . . . .
. . . B B . . .
P Q P . N P r P
R . . . . R K .


In [57]:
engine = chess.engine.SimpleEngine.popen_uci(engine_location)
print(this_board)
#print(str(engine.analyse(this_board, chess.engine.Limit(depth=0))["score"]))
#this_board.push_san("Ba3")
#this_board.push_san("d6")
#this_board.push_san("Rad1")
#this_board.push_san("Qd1")
#this_board.push_san("Re1")
#this_board.push_san("Kh2")




print(str(engine.analyse(this_board, chess.engine.Limit(depth=1))["score"]))

print(str(engine.analyse(this_board, chess.engine.Limit(depth=10))["score"]))

print(this_board)

print(get_top_moves(this_board, 1))

print("\n")
print(solutions[z])
print(this_board.fen())
engine.quit()

#this_board.pop()
#this_board.pop()
#this_board.pop()
#this_board.pop()

. . k r . b . .
p p p q . p . p
. . n . b p . .
. . . . . . . .
. . . P . . . .
. . . B B . . .
P Q P . N P r P
R . . . . R K .
+342
+369
. . k r . b . .
p p p q . p . p
. . n . b p . .
. . . . . . . .
. . . P . . . .
. . . B B . . .
P Q P . N P r P
R . . . . R K .
[594, -369] ['g1h1', 'g1g2']
([594, -369], ['g1h1', 'g1g2'])


['Kxg2', 'Bh3+', 'Kg1', 'Qd5', 'f3']
2kr1b2/pppq1p1p/2n1bp2/8/3P4/3BB3/PQP1NPrP/R4RK1 w - - 0 2


In [242]:
this_board.pop()

Move.from_uci('c1a3')